Machine Translation for Persian (Persian-English or PersianOther Language)(Mr. Momeni)
• Objective: Build a machine translation system for Persian to
another language (English).
• Tasks:
• Use a parallel corpus of Persian and the target language.
• Train a Seq2Seq model with attention or a transformer-based
model (e.g., MarianMT).
• Fine-tune an existing model pretrained on multilingual
corpora.
• Evaluate translation quality using BLEU or METEOR scores.
• Deliverable: A report on the translation model, training process,
and evaluation results

# ***import library and neccsary file from git***

In [1]:
!git clone https://github.com/omidkashefi/Mizan.git

fatal: destination path 'Mizan' already exists and is not an empty directory.


In [2]:
pip install hazm

In [3]:
pip install hazm transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
pip install torch transformers

In [5]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00


In [6]:
pip install torch pytorch-lightning hazm nltk tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 45.5 MB/s eta 0:00:00


In [8]:
import zipfile
import os
from hazm import *
import nltk
import torch
import random
from nltk.tokenize import sent_tokenize, word_tokenize
import time
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from tqdm import tqdm
import sacrebleu
import re
from hazm import Normalizer
import collections

In [9]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# ***Read dataset and preapre for feeding to model***

In [10]:
def unzip_file(zip_file_path, dest_directory):
    # Create the destination directory if it doesn't exist
    os.makedirs(dest_directory, exist_ok=True)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dest_directory)

zip_file_path = '/content/Mizan/mizan.zip'
dest_directory = '/content/'
unzip_file(zip_file_path, dest_directory)

In [11]:
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read().splitlines()
    except FileNotFoundError:
        return []
    except IOError:
        return []

file_path_en = '/content/mizan/mizan_en.txt'
file_path_fa = '/content/mizan/mizan_fa.txt'

english_sentences = read_text_file(file_path_en)
persian_sentences = read_text_file(file_path_fa)

assert len(english_sentences) == len(persian_sentences), "The corpus files are not aligned."

In [12]:
# Initialize the hazm normalizer
normalizer = Normalizer()

def preprocess_persian(text):
    # Normalize the text
    normalized_text = normalizer.normalize(text)
    sentences = sent_tokenize(normalized_text)
    tokens = [word_tokenize(sentence) for sentence in sentences]
    return tokens

# Process
example_persian = persian_sentences[0]
print("Original:", example_persian)
print("Tokenized:", preprocess_persian(example_persian))


Original: داستانی که از نظر شما می‌گذرد، ابتدا ضمن کنفرانس صلح پاریس از روی یادداشت‌هائی که به طور روزانه در حال خدمت در صف برداشته شده بودند
Tokenized: [['داستانی', 'که', 'از', 'نظر', 'شما', 'می\u200cگذرد،', 'ابتدا', 'ضمن', 'کنفرانس', 'صلح', 'پاریس', 'از', 'روی', 'یادداشت\u200cهائی', 'که', 'به', 'طور', 'روزانه', 'در', 'حال', 'خدمت', 'در', 'صف', 'برداشته\u200cشده', 'بودند']]


In [13]:
def preprocess_english(text):
    sentences = sent_tokenize(text)
    tokens = [word_tokenize(sentence) for sentence in sentences]
    return tokens

# Process
example_english = english_sentences[0]
print("Original:", example_english)
print("Tokenized:", preprocess_english(example_english))

Original: The story which follows was first written out in Paris during the Peace Conference
Tokenized: [['The', 'story', 'which', 'follows', 'was', 'first', 'written', 'out', 'in', 'Paris', 'during', 'the', 'Peace', 'Conference']]


In [14]:
# Build the dataset as a list of (persian_tokens, english_tokens) pairs.
pairs = []
# Assume that the sentences are aligned in order.
num_examples = min(len(english_sentences), len(persian_sentences))
for i in range(num_examples):
    eng_tokens = preprocess_english(english_sentences[i])
    fa_tokens = preprocess_persian(persian_sentences[i])
    # Discard any pair with an empty token list
    if eng_tokens and fa_tokens:
        pairs.append((fa_tokens, eng_tokens))

print(f"\nTotal sentence pairs: {len(pairs)}")


Total sentence pairs: 1021594


# ***build transformer foe seq2seq translation***

In [15]:
# Use only 2% of the full data for this demo "BECUSE I DONT HAVE HAEDWER":
# subset_size = int(len(pairs))
subset_size = int(0.02 * len(pairs))
subset_pairs = pairs[:subset_size]
print("Using 2% of data:", len(subset_pairs))

# Split the subset into train (80%), validation (10%), and test (10%) sets.
total_subset = len(subset_pairs)
train_end = int(0.8 * total_subset)
valid_end = int(0.9 * total_subset)
train_pairs = subset_pairs[:train_end]
valid_pairs = subset_pairs[train_end:valid_end]
test_pairs  = subset_pairs[valid_end:]
print("Train pairs:", len(train_pairs))
print("Validation pairs:", len(valid_pairs))
print("Test pairs:", len(test_pairs))

# ---------------------------
# 2. Build Vocabularies and Helper Functions
# ---------------------------
# Special tokens
PAD_TOKEN = '<pad>'
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

def flatten(list_of_lists):
    return [token for sublist in list_of_lists for token in sublist]

def build_vocab(tokenized_sentences):
    vocab = {PAD_TOKEN: 0, SOS_TOKEN: 1, EOS_TOKEN: 2}
    idx = 3
    for tokens in tokenized_sentences:
        for token in tokens:
            if token not in vocab:
                vocab[token] = idx
                idx += 1
    return vocab

# Build vocabularies using the training set.
src_vocab = build_vocab([flatten(pair[0]) for pair in train_pairs])
trg_vocab = build_vocab([flatten(pair[1]) for pair in train_pairs])
print("Source (Persian) vocab size:", len(src_vocab))
print("Target (English) vocab size:", len(trg_vocab))

def tokens_to_tensor(tokens, vocab):
    flat_tokens = flatten(tokens) if any(isinstance(t, list) for t in tokens) else tokens
    indices = [vocab[SOS_TOKEN]] + [vocab[token] for token in flat_tokens if token in vocab] + [vocab[EOS_TOKEN]]
    return torch.tensor(indices, dtype=torch.long)

# ---------------------------
# 3. Create Dataset and DataLoader
# ---------------------------
class TranslationDataset(Dataset):
    def __init__(self, pairs, src_vocab, trg_vocab):
        self.pairs = pairs
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        fa_tokens, eng_tokens = self.pairs[idx]
        src_tensor = tokens_to_tensor(fa_tokens, self.src_vocab)
        trg_tensor = tokens_to_tensor(eng_tokens, self.trg_vocab)
        return src_tensor, trg_tensor

def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=src_vocab[PAD_TOKEN])
    trg_batch = pad_sequence(trg_batch, padding_value=trg_vocab[PAD_TOKEN])
    return src_batch, trg_batch

BATCH_SIZE = 32
train_dataset = TranslationDataset(train_pairs, src_vocab, trg_vocab)
valid_dataset = TranslationDataset(valid_pairs, src_vocab, trg_vocab)
test_dataset  = TranslationDataset(test_pairs, src_vocab, trg_vocab)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# ---------------------------
# 4. Define the Seq2Seq Model with Attention
# ---------------------------
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, num_layers, dropout):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hid_dim*2, hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        hidden_forward = hidden[-2, :, :]
        hidden_backward = hidden[-1, :, :]
        hidden = torch.tanh(self.fc(torch.cat((hidden_forward, hidden_backward), dim=1)))
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(enc_hid_dim*2 + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        src_len = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        return torch.softmax(attention, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, num_layers, dropout, attention):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(enc_hid_dim*2 + emb_dim, dec_hid_dim, num_layers, dropout=dropout)
        self.fc_out = nn.Linear(enc_hid_dim*2 + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1,0,2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1,0,2)
        rnn_input = torch.cat((embedded, weighted), dim=2)
        hidden = hidden.unsqueeze(0)
        batch_size = hidden.shape[1]
        cell = torch.zeros_like(hidden).to(hidden.device)
        output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        output = output.squeeze(0)
        embedded = embedded.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        return prediction, hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        input = trg[0, :]
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs

# ---------------------------
# 5. Define a LightningModule for Training
# ---------------------------
class Seq2SeqModule(pl.LightningModule):
    def __init__(self, model, lr, trg_pad_idx, teacher_forcing_ratio=0.5):
        super(Seq2SeqModule, self).__init__()
        self.model = model
        self.lr = lr
        self.trg_pad_idx = trg_pad_idx
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

    def forward(self, src, trg):
        return self.model(src, trg, teacher_forcing_ratio=self.teacher_forcing_ratio)

    def training_step(self, batch, batch_idx):
        src, trg = batch
        output = self(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = self.criterion(output, trg)

        preds = output.argmax(dim=1)
        correct = (preds == trg).float().sum()
        total = (trg != self.trg_pad_idx).float().sum()
        acc = correct / total if total > 0 else 0

        # Get the actual batch size.
        batch_size = src.size(1)

        # Log with an explicit batch_size.
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True, batch_size=batch_size)
        self.log("train_acc", acc, prog_bar=True, on_step=True, on_epoch=True, batch_size=batch_size)
        return loss


    def validation_step(self, batch, batch_idx):
        src, trg = batch
        output = self(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = self.criterion(output, trg)

        preds = output.argmax(dim=1)
        correct = (preds == trg).float().sum()
        total = (trg != self.trg_pad_idx).float().sum()
        acc = correct / total if total > 0 else 0

        batch_size = src.size(1)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, batch_size=batch_size)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True, batch_size=batch_size)
        return loss


    def test_step(self, batch, batch_idx):
        src, trg = batch
        output = self(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = self.criterion(output, trg)
        preds = output.argmax(dim=1)
        correct = (preds == trg).float().sum()
        total = (trg != self.trg_pad_idx).float().sum()
        acc = correct / total if total > 0 else 0
        self.log("test_loss", loss, prog_bar=True, on_epoch=True)
        self.log("test_acc", acc, prog_bar=True, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        return optimizer

# ---------------------------
# 6. Initialize Model and Trainer
# ---------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
NUM_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, NUM_LAYERS, ENC_DROPOUT)
attention = Attention(HID_DIM, HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, HID_DIM, NUM_LAYERS, DEC_DROPOUT, attention)
seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)

# Create the LightningModule
model_module = Seq2SeqModule(seq2seq_model, lr=1e-3, trg_pad_idx=trg_vocab[PAD_TOKEN], teacher_forcing_ratio=0.5)

# Setup callbacks
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1)
progress_bar = TQDMProgressBar(refresh_rate=50)

#  """"""NUMBER OF EPOCH BARAYE IN KAME K RUNTIME BISHTAR AZ 2 SAAT NMMIZARE AZ GPU STEFADE KONAM"""""""""""

# Initialize the PyTorch Lightning Trainer.
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    callbacks=[checkpoint_callback, progress_bar],
    precision=16  # Mixed precision to speed up training
)

# ---------------------------
# 7. Train and Test
# ---------------------------
trainer.fit(model_module, train_loader, valid_loader)
trainer.test(model_module, test_loader)

Using 2% of data: 20431
Train pairs: 16344
Validation pairs: 2043
Test pairs: 2044
Source (Persian) vocab size: 24649
Target (English) vocab size: 18044
Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/lightning_fabric/connector.py:572: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-----------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 35. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 26. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 31. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 23. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/loc

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.14214882254600525    │
│         test_loss         │    6.5991315841674805     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 6.5991315841674805, 'test_acc': 0.14214882254600525}]

# ***Out put test of model***

In [16]:
# Assume trg_vocab and src_vocab are defined as before.
# Build an inverse mapping for the target vocabulary.
trg_ivocab = {idx: token for token, idx in trg_vocab.items()}

def translate_sentence(sentence, src_vocab, trg_vocab, model, device, max_len=50):
    """
    Translate a source sentence using the trained Seq2Seq model.

    Args:
        sentence (str): Input sentence in the source language.
        src_vocab (dict): Vocabulary for the source language.
        trg_vocab (dict): Vocabulary for the target language.
        model (Seq2Seq): The trained Seq2Seq model.
        device (torch.device): The device (CPU or GPU) to run on.
        max_len (int): Maximum number of tokens to generate.

    Returns:
        str: The translated sentence.
    """
    model.eval()

    # Preprocess the input sentence (assume it's Persian)
    tokens = preprocess_persian(sentence)

    # Convert tokens to tensor and move to device.
    # tokens_to_tensor adds <sos> and <eos>.
    src_tensor = tokens_to_tensor(tokens, src_vocab).unsqueeze(1).to(device)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

    # Initialize the decoder input with the <sos> token.
    input_token = torch.tensor([trg_vocab[SOS_TOKEN]], device=device)

    translated_tokens = []

    for _ in range(max_len):
        with torch.no_grad():
            output, hidden = model.decoder(input_token, hidden, encoder_outputs)

        # Get the index of the highest probability token.
        pred_token = output.argmax(dim=1).item()

        # Stop if <eos> is generated.
        if pred_token == trg_vocab[EOS_TOKEN]:
            break

        translated_tokens.append(pred_token)

        # Set the predicted token as the next input.
        input_token = torch.tensor([pred_token], device=device)

    # Convert token indices to words.
    translated_words = [trg_ivocab[idx] for idx in translated_tokens]
    translated_sentence = ' '.join(translated_words)
    return translated_sentence

# ---------------------------
# Example Usage:
# ---------------------------

# Create a proper device object.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure your model is moved to that device.
seq2seq_model.to(device)

sample_sentence = "داستانی که از نظر شما می‌گذرد، ابتدا ضمن کنفرانس صلح پاریس از روی یادداشت‌ها نوشته شد."

translated = translate_sentence(
    sample_sentence,
    src_vocab,
    trg_vocab,
    seq2seq_model,
    device=device,  # Pass the device object here
    max_len=50
)

print("Input Sentence:", sample_sentence)
print("Translated Sentence:", translated)

Input Sentence: داستانی که از نظر شما می‌گذرد، ابتدا ضمن کنفرانس صلح پاریس از روی یادداشت‌ها نوشته شد.
Translated Sentence: The story of follows , the follows of Paris during the Peace Conference of Peace Conference .


# **Python Code to Combine and Preprocess Files dataset of teacher**

In [ ]:
data_dir = "/content/data"

normalizer = Normalizer()

def preprocess_text(text):

    text = normalizer.normalize(text)

    text = re.sub(r'\s+', ' ', text)

    text = text.strip()
    return text

combined_text = ""
for i in range(1, 32):
    file_path = f"{data_dir}/{i}.txt"
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        processed_text = preprocess_text(text)
        combined_text += processed_text

combined_output = "combined_preprocessed.txt"
with open(combined_output, "w", encoding="utf-8") as outfile:
    outfile.write(combined_text)

print(f"Combined and preprocessed text saved to {combined_output}")


Combined and preprocessed text saved to combined_preprocessed.txt


In [ ]:
input_file_path = "/content/combined_preprocessed.txt"

with open(input_file_path, "r", encoding="utf-8") as file:
    text = file.read()

char_counter = collections.Counter(text)

print("Unique characters and their counts:\n")
for char, count in sorted(char_counter.items(), key=lambda x: x[0]):
    if char == " ":
        display_char = "[SPACE]"
    elif char == "\n":
        display_char = "[NEWLINE]"
    elif char == "\t":
        display_char = "[TAB]"
    elif char == "\u200c":
        display_char = "[ZWNJ]"
    elif char == "\u200f":
        display_char = "[RLM]"
    else:
        display_char = char
    print(f"{display_char!r}: {count}")

unwanted_chars = {"[", "]", "a", "c", "h", "k", "l", "n", "\u200f", "@", "#", "$", "%", "^", "&"}

processed_text = ''.join(char for char in text if char not in unwanted_chars)

replacement_map = {
    "أ": "ا",
    "ؤ": "و",
    "ئ": "ی",
    "ة": "ت"
}

processed_text = processed_text.replace("ء", "")

for old_char, new_char in replacement_map.items():
    processed_text = processed_text.replace(old_char, new_char)

processed_text = re.sub(r'\s+', ' ', processed_text).strip()

cleaned_file_path = "/content/cleaned_combined_preprocessed3.txt"
with open(cleaned_file_path, "w", encoding="utf-8") as outfile:
    outfile.write(processed_text)

print("\nCleaned and updated text saved to:", cleaned_file_path)


Unique characters and their counts:

'[SPACE]': 224169
'[': 183
']': 183
'a': 3
'c': 18
'h': 8
'k': 17
'l': 1
'n': 5
'ء': 10
'آ': 5665
'أ': 376
'ؤ': 193
'ئ': 797
'ا': 111474
'ب': 42202
'ة': 3
'ت': 36254
'ث': 2169
'ج': 8715
'ح': 8524
'خ': 12576
'د': 55136
'ذ': 1117
'ر': 52703
'ز': 12486
'س': 19080
'ش': 23959
'ص': 4408
'ض': 2186
'ط': 3888
'ظ': 1331
'ع': 11128
'غ': 907
'ف': 12218
'ق': 12842
'ل': 22836
'م': 65392
'ن': 67325
'ه': 59531
'و': 56703
'پ': 3372
'چ': 5323
'ژ': 588
'ک': 26732
'گ': 11288
'ی': 91806
'۰': 409
'۱': 321
'۲': 265
'۳': 189
'۴': 201
'۵': 270
'۶': 179
'۷': 169
'۸': 341
'۹': 153
'[ZWNJ]': 9832
'[RLM]': 52

Cleaned and updated text saved to: /content/cleaned_combined_preprocessed3.txt


In [ ]:
input_file_path = "/content/cleaned_combined_preprocessed3.txt"

with open(input_file_path, "r", encoding="utf-8") as file:
    text = file.read()

char_counter = collections.Counter(text)

print("Unique characters and their counts:\n")
for char, count in sorted(char_counter.items(), key=lambda x: x[0]):
    if char == " ":
        display_char = "[SPACE]"
    elif char == "\n":
        display_char = "[NEWLINE]"
    elif char == "\t":
        display_char = "[TAB]"
    elif char == "\u200c":
        display_char = "[ZWNJ]"
    elif char == "\u200f":
        display_char = "[RLM]"
    else:
        display_char = char
    print(f"{display_char!r}: {count}")

Unique characters and their counts:

'[SPACE]': 224120
'آ': 5665
'ا': 111850
'ب': 42202
'ت': 36257
'ث': 2169
'ج': 8715
'ح': 8524
'خ': 12576
'د': 55136
'ذ': 1117
'ر': 52703
'ز': 12486
'س': 19080
'ش': 23959
'ص': 4408
'ض': 2186
'ط': 3888
'ظ': 1331
'ع': 11128
'غ': 907
'ف': 12218
'ق': 12842
'ل': 22836
'م': 65392
'ن': 67325
'ه': 59531
'و': 56896
'پ': 3372
'چ': 5323
'ژ': 588
'ک': 26732
'گ': 11288
'ی': 92603
'۰': 409
'۱': 321
'۲': 265
'۳': 189
'۴': 201
'۵': 270
'۶': 179
'۷': 169
'۸': 341
'۹': 153
'[ZWNJ]': 9832


In [17]:
import sacrebleu

hypotheses = [
    """input u data """
]

references = [
    """input model translation"""
]

bleu = sacrebleu.corpus_bleu(hypotheses, references)
print("BLEU score:", bleu.score)


BLEU score: 27.901593935858266
